# Notebook de modelisation

In [9]:
import pandas as pd

# Import des dataframes RFM

In [10]:
rfm = pd.read_csv('../data/dataset_rfm.csv')
rfm.head()

,Unnamed: 0,customer_unique_id,order_purchase_timestamp,R,F,M
0,0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,160,1,141.90
1,1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,163,1,27.19
2,2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,585,1,86.22
3,3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,369,1,43.62
4,4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,336,1,196.89


In [11]:
rfmls  = pd.read_csv('../data/dataset_rfm_ls.csv')
rfmls.head()

,Unnamed: 0,customer_unique_id,order_purchase_timestamp,R,F,M,S,L
0,0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,160,1,141.90,5,cajamar
1,1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,163,1,27.19,4,osasco
2,2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,585,1,86.22,3,sao jose
3,3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,369,1,43.62,4,belem
4,4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,336,1,196.89,5,sorocaba


# Benchmark des models

In [12]:
# from pycaret.clustering import setup, create_model, ClusteringExperiment, pull

# # Initialisation de l'environnement PyCaret pour le clustering
# exp_clu = setup(
#     data=rfm.dropna(), 
#     use_gpu = True,
#     ignore_features= ['customer_unique_id', 'order_purchase_timestamp'],
#     preprocess=True, 
#     experiment_name="benchmark_clustering_model"
#     )

In [13]:
# def model_testing(modelname):
#     print("---------------------------")
#     dbscan = create_model(modelname)
#     pull(dbscan)
#     print(dbscan)
#     print("---#---#---#---#---#---#---")
    

In [14]:
model_list = ['dbscan', 'optics', 'meanshift','ap', 'sc', 'hclust', 'kmeans']

In [15]:
# import pandas as pd
# import numpy as np
# from sklearn import cluster, metrics
# from sklearn.datasets import make_blobs
# import time
# import hdbscan  # Pour HDBSCAN


# rfm = rfm.drop(['customer_unique_id', 'order_purchase_timestamp'], axis=1).dropna()
# rfm_sample = rfm.sample(frac=0.1, random_state=42)  # Prend 10% des données
# print(rfm_sample.shape)
# # Liste des algorithmes à comparer
# model_list = {
#     'dbscan': cluster.DBSCAN(eps=0.5, min_samples=5),
#     'optics': cluster.OPTICS(min_samples=5),
#     'meanshift': cluster.MeanShift(),
#     'ap': cluster.AffinityPropagation(),
#     'sc': cluster.SpectralClustering(n_clusters=3, assign_labels='discretize'),
#     'hclust': cluster.AgglomerativeClustering(n_clusters=3),
#     'kmeans': cluster.KMeans(n_clusters=3),
#     'hdbscan':hdbscan.HDBSCAN(min_cluster_size=5)
# }

# # Tableau pour stocker les résultats
# results = []

# # Évaluation des algorithmes
# for name, algorithm in model_list.items():
#     start_time = time.time()
    
#     # Fit et prédire les labels
#     labels = algorithm.fit_predict(rfm_sample)
#     elapsed_time = time.time() - start_time
    
#     # Calcul des scores
#     silhouette_score = metrics.silhouette_score(rfm_sample,labels)if len(set(labels)) > 1 else None
#     davies_bouldin_score = metrics.davies_bouldin_score(rfm_sample,labels) if len(set(labels)) > 1 else None
#     calinski_harabasz_score = metrics.calinski_harabasz_score(rfm_sample,labels) if len(set(labels)) > 1 else None

#     results.append({
#         "Algorithm": name,
#         "Time (s)": elapsed_time,
#         "Silhouette Score": silhouette_score,
#         "Davies-Bouldin Score": davies_bouldin_score,
#         "Calinski-Harabasz Score": calinski_harabasz_score
#     })

# # Création d'un DataFrame pour afficher les résultats
# results_df = pd.DataFrame(results)
# print(results_df)


In [16]:
import pandas as pd
import numpy as np
import time
from joblib import Parallel, delayed
from sklearn import cluster, metrics
import hdbscan  # Pour HDBSCAN

from sklearn.preprocessing import StandardScaler

from sklearn import cluster
from sklearn.mixture import GaussianMixture
from sklearn_extra.cluster import KMedoids
import hdbscan
import skfuzzy as fuzz  # Pour Fuzzy C-Means

# Chargement des données (exemple)
# rfm = pd.read_csv("data.csv")  # Décommenter pour charger un dataset réel

# Suppression de colonnes non pertinentes et valeurs manquantes
rfm = rfm.drop(['customer_unique_id', 'order_purchase_timestamp'], axis=1).dropna()

# Échantillonnage si le dataset est trop grand (> 10 000 lignes)
if rfm.shape[0] > 10000:
    rfm_sample = rfm.sample(frac=0.1, random_state=42)
else:
    rfm_sample = rfm


# if np.isnan(rfm_sample).any() or np.isinf(rfm_sample).any():
#     print("Attention : NaN ou Inf détectés dans les données !")




scaler = StandardScaler()
rfm_sample_scaled = scaler.fit_transform(rfm_sample)


In [17]:

# Liste des algorithmes à comparer (avec optimisation)
model_list = {
    # Basés sur la densité
    'dbscan': cluster.DBSCAN(eps=0.5, min_samples=5),
    'optics': cluster.OPTICS(min_samples=5),
    'hdbscan': hdbscan.HDBSCAN(min_cluster_size=5),

    # Basés sur les centroïdes
    'kmeans': cluster.KMeans(n_clusters=3, n_init='auto'),
    'minibatch_kmeans': cluster.MiniBatchKMeans(n_clusters=3, n_init='auto'),
    'kmedoids': KMedoids(n_clusters=3, random_state=42),

    # Basés sur la hiérarchie
    'hclust': cluster.AgglomerativeClustering(n_clusters=3),
    'birch': cluster.Birch(n_clusters=3),

    # Basés sur les graphes
    'spectral': cluster.SpectralClustering(n_clusters=3, assign_labels='discretize'),
    'ap': cluster.AffinityPropagation(),

    # Basés sur la distribution
    'gmm': GaussianMixture(n_components=3, random_state=42)
}

In [ ]:
#  Limiter certains algorithmes coûteux si le dataset est petit
if rfm_sample_scaled.shape[0] < 5000:
    model_list.update({
        'meanshift': cluster.MeanShift(),
        'ap': cluster.AffinityPropagation(),
        'sc': cluster.SpectralClustering(n_clusters=3, assign_labels='discretize')
    })

# Fonction d'évaluation des algorithmes
def evaluate_algorithm(name, algorithm, rfm):
    print(f"Exécution de {name}...")
    start_time = time.time()

    # Fit et prédire les labels
    labels = algorithm.fit_predict(rfm)
    elapsed_time = time.time() - start_time

    # Vérification du nombre de clusters
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

    if n_clusters > 1:
        silhouette_score = metrics.silhouette_score(rfm, labels)
        davies_bouldin_score = metrics.davies_bouldin_score(rfm, labels)
        calinski_harabasz_score = metrics.calinski_harabasz_score(rfm, labels)
    else:
        silhouette_score = davies_bouldin_score = calinski_harabasz_score = None

    return {
        "Algorithm": name,
        "Time (s)": round(elapsed_time, 3),
        "Clusters": n_clusters,
        "Silhouette Score": silhouette_score,
        "Davies-Bouldin Score": davies_bouldin_score,
        "Calinski-Harabasz Score": calinski_harabasz_score
    }

# Exécution parallèle des algorithmes
results = Parallel(n_jobs=-1)(delayed(evaluate_algorithm)(name, algo, rfm_sample_scaled) for name, algo in model_list.items())

# Création du DataFrame des résultats
results_df = pd.DataFrame(results)

# Affichage des résultats triés par Silhouette Score
results_df.sort_values(by="Silhouette Score", ascending=False, na_position='last')